In [1]:
import pandas as pd
import os

In [ ]:
"""
코드 작성자 : DevTae (김태현)
"""

"""
데이터 전처리 순서 (작성자 : DevTae (김태현))

1. Dart (재무정보), KRX (가격정보) 를 바탕으로 데이터를 수집한다.
2-1. Dart 에서의 재무 데이터를 'year_quarter/pre_data/code_quarter.csv' 형식으로 저장한다.
     ex. 2020_1/pre_data/000020_1.csv
     (담당자 : 김민준)
2-2. KRX 에서 수정한 주가 데이터를 바탕으로 ATR, TSF_SLOPE, MARKETCAP 에 대하여 연산하고, 
     'price_stat_datas/code_year_quarter.csv' 형식으로 저장한다.
     ex. price_stat_datas/000020_2020_1.csv
     (담당자 : 김태현)
3. 위 2-1, 2-2 에 대한 데이터를 하나의 데이터로 병합한다.
   해당 데이터는 'data_before/finance_datas/code_year_quarter.csv' 파일에 Pandas 형식으로 저장한다.
4. 하나의 데이터로 합친 Pandas 데이터에서 사용하는 지표를 가져오고, 추가 지표를 정의한 뒤,
   전처리 완료된 파일을 data 폴더에 저장한다.
   - data_preprocessing_unified.csv : 사용 지표만 불러온 파일
   - data_preprocessing_unified_including_others.csv : 위 파일에 추가 지표까지 포함한 파일
   - data_preprocessing_unified_including_others_and_percentile.csv : 위 파일에 백분위 정보까지 포함한 파일
5. 이후, data 폴더의 데이터를 바탕으로 데이터 탐색 및 모델링에 활용한다.

2-1번 과정은 data_preprocessing_1_get_financial.ipynb 에서 진행하며,
2-2번 과정은 별도의 프로젝트를 바탕으로 데이터를 수집하였습니다.
3번 과정은 data_preprocessing_2_concat.ipynb 에서 2-1, 2-2 데이터의 병합을 진행하고,
4번 과정은 data_preproecssing_3_additional.ipynb (해당 파일) 에서 진행한다.

"""

In [5]:
# 각 데이터들을 모아주는 함수 역할 (data/data_preprocessing_unified.csv 없을 때 실행)

dataset_path = 'data_before/concat' # 병합된 데이터를 바탕으로 선별 및 재구성 진행

list_of_directory = [ "2020_1_concat/1_concat",
                      "2020_2_concat/2_concat",
                      "2020_3_concat/3_concat",
                      "2020_4_concat/4_concat",
                      "2021_1_concat/1_concat",
                      "2021_2_concat/2_concat",
                      "2021_3_concat/3_concat",
                      "2021_4_concat/4_concat",
                      "2022_1_concat/1_concat",
                      "2022_2_concat/2_concat",
                      "2022_3_concat/3_concat",
                      "2022_4_concat/4_concat",
                      "2023_1_concat/1_concat",
                      "2023_2_concat/2_concat",
                      "2023_3_concat/3_concat" ]

finance_datas = pd.DataFrame()

for directory in list_of_directory:

    files_of_finance_datas = os.listdir(os.path.join(dataset_path, directory))
    using_columns = [ '배당금지급', '유동자산', '유동부채', '영업이익(손실)', '수익(매출액)',\
                      '당기순이익(손실)', '자본총계', '자산총계', '부채총계', 'ATR', 'MARKETCAP', 'TSF_SLOPE' ]

    year = directory.split('_')[0]
    quarter = directory.split('_')[1]
    
    for filename in files_of_finance_datas:
        if '.csv' not in filename:
            continue
        
        code = filename.split('_')[0]
        #year = filename.split('_')[1]
        #quarter = filename.split('_')[2].split('.')[0]
        
        #csv_data = pd.read_csv(os.path.join(dataset_path, filename), encoding='ANSI', skipinitialspace=True)
        csv_data = pd.read_csv(os.path.join(dataset_path, directory, filename), encoding='euc-kr', skipinitialspace=True)
        
        finance_data = { }
        
        finance_data['종목코드'] = str(code)
        finance_data['년도'] = int(year)
        finance_data['분기'] = int(quarter)
        
        skip = False
        
        for using_column in using_columns:
            if using_column not in csv_data.columns:
                skip = True
                break
            finance_data[using_column] = csv_data[using_column].item()
            
        if skip == True:
            continue
            
        finance_datas = finance_datas.append(finance_data, ignore_index=True)
    
try:
    os.mkdir('data')
except:
    pass

finance_datas.to_csv('data/data_preprocessing_unified.csv', sep=',', encoding='utf-8', index=False)

finance_datas

,종목코드,년도,분기,배당금지급,유동자산,유동부채,영업이익(손실),수익(매출액),당기순이익(손실),자본총계,자산총계,부채총계,ATR,MARKETCAP,TSF_SLOPE
0,000020,2020.0,1.0,3.312928e+09,2.379160e+11,5.601470e+10,2.706892e+09,6.700367e+10,2.259000e+09,2.994870e+11,3.761860e+11,7.669876e+10,0.040,1.829511e+11,-0.332
1,000950,2020.0,1.0,0.000000e+00,4.445098e+10,2.139210e+11,-3.642227e+09,2.840698e+10,-7.534318e+09,1.237910e+11,3.870420e+11,2.632520e+11,0.043,1.940399e+10,-0.329
2,001540,2020.0,1.0,0.000000e+00,1.473480e+11,5.060352e+10,2.657248e+09,3.777548e+10,2.329921e+09,1.491430e+11,2.130380e+11,6.389562e+10,0.048,1.237726e+11,-0.310
3,001790,2020.0,1.0,6.212995e+09,5.346720e+11,4.020430e+11,7.324572e+09,2.836740e+11,1.456410e+08,4.575730e+11,1.153100e+12,6.955260e+11,0.038,1.412721e+11,-0.184
4,002230,2020.0,1.0,0.000000e+00,9.711871e+10,2.172407e+10,-3.697193e+08,1.134262e+10,-2.315460e+09,1.336150e+11,1.600970e+11,2.648246e+10,0.046,5.249453e+10,-0.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,383220,2023.0,3.0,6.102839e+10,7.170640e+11,4.755630e+11,1.485240e+11,4.928150e+11,1.190810e+11,1.221050e+12,1.784690e+12,5.636410e+11,0.039,4.328701e+12,0.004
3342,405100,2023.0,3.0,5.737761e+09,7.099587e+10,8.948139e+09,1.729899e+09,1.500569e+10,1.754362e+09,1.018220e+11,1.304090e+11,2.858691e+10,0.045,1.580033e+11,0.058
3343,425040,2023.0,3.0,4.249243e+09,1.721390e+11,6.219917e+10,3.685606e+09,2.418480e+10,4.575588e+09,1.681180e+11,2.394000e+11,7.128183e+10,0.052,4.170749e+11,-0.301
3344,900070,2023.0,3.0,4.103465e+06,5.672710e+08,1.290003e+08,9.139425e+06,1.489769e+08,4.868978e+06,6.494237e+08,8.367302e+08,1.873065e+08,0.040,3.342875e+10,-0.169


In [20]:
# 데이터 로딩 진행 (data/data_preprocessing_unified.csv 불러올 때 실행)

finance_datas = pd.read_csv('data/data_preprocessing_unified.csv', sep=',', encoding='utf-8', dtype={'종목코드':str, '분기':int})

finance_datas

,종목코드,년도,분기,배당금지급,유동자산,유동부채,영업이익(손실),수익(매출액),당기순이익(손실),자본총계,자산총계,부채총계,ATR,MARKETCAP,TSF_SLOPE
0,000020,2020.0,1,3.312928e+09,2.379160e+11,5.601470e+10,2.706892e+09,6.700367e+10,2.259000e+09,2.994870e+11,3.761860e+11,7.669876e+10,0.040,1.829511e+11,-0.332
1,000950,2020.0,1,0.000000e+00,4.445098e+10,2.139210e+11,-3.642227e+09,2.840698e+10,-7.534318e+09,1.237910e+11,3.870420e+11,2.632520e+11,0.043,1.940399e+10,-0.329
2,001540,2020.0,1,0.000000e+00,1.473480e+11,5.060352e+10,2.657248e+09,3.777548e+10,2.329921e+09,1.491430e+11,2.130380e+11,6.389562e+10,0.048,1.237726e+11,-0.310
3,001790,2020.0,1,6.212995e+09,5.346720e+11,4.020430e+11,7.324572e+09,2.836740e+11,1.456410e+08,4.575730e+11,1.153100e+12,6.955260e+11,0.038,1.412721e+11,-0.184
4,002230,2020.0,1,0.000000e+00,9.711871e+10,2.172407e+10,-3.697193e+08,1.134262e+10,-2.315460e+09,1.336150e+11,1.600970e+11,2.648246e+10,0.046,5.249453e+10,-0.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,383220,2023.0,3,6.102839e+10,7.170640e+11,4.755630e+11,1.485240e+11,4.928150e+11,1.190810e+11,1.221050e+12,1.784690e+12,5.636410e+11,0.039,4.328701e+12,0.004
3342,405100,2023.0,3,5.737761e+09,7.099587e+10,8.948139e+09,1.729899e+09,1.500569e+10,1.754362e+09,1.018220e+11,1.304090e+11,2.858691e+10,0.045,1.580033e+11,0.058
3343,425040,2023.0,3,4.249243e+09,1.721390e+11,6.219917e+10,3.685606e+09,2.418480e+10,4.575588e+09,1.681180e+11,2.394000e+11,7.128183e+10,0.052,4.170749e+11,-0.301
3344,900070,2023.0,3,4.103465e+06,5.672710e+08,1.290003e+08,9.139425e+06,1.489769e+08,4.868978e+06,6.494237e+08,8.367302e+08,1.873065e+08,0.040,3.342875e+10,-0.169


In [21]:
# 데이터 추가 입력

"""
1. 변동성 지표 (Slow vs Fast) - 클수록 Fast
 - ATR : ATRPP (Average True Range Per Price)

2. 수익성 지표 (Poor vs Rich) - 클수록 Rich
 - 배당수익률
 - 순이익률 (Net Profit Margin)
 - 자본회전율 (Asset Turnover)
 
3. 안정성 지표 (Danger vs Safety) - 클수록 Safety
 - Altman Z Score
   - Z > 2.99 : 안정적 상태
   - 1.81 < Z < 2.99 : 주의 요망
   - Z < 1.81 : 파산 가능성이 높음
 - 부채비율 : 부채비율이 커지면 Danger 하기 때문에 전처리 과정에서 (-1) 를 곱해줌.

4. 가치성 지표 (Hidden vs Popular) - 클수록 Popular
 - PER (시가총액 / 순이익)
 - PSR (시가총액 / 매출)
 - PBR (시가총액 / 순자산)

5. 모멘텀 지표 (Weak vs Strong) - 클수록 Strong
 - TSF Slope : TSF (Time Series Forcasting) 지표에 대한 Slope (기울기) 계산 (Percentage 방식)
"""
    
finance_datas['배당수익률'] = finance_datas['배당금지급'] / finance_datas['MARKETCAP']
finance_datas['순이익률'] = finance_datas['당기순이익(손실)'] / finance_datas['수익(매출액)']
finance_datas['자본회전율'] = finance_datas['수익(매출액)'] / finance_datas['자본총계']

finance_datas['AltmanZScore'] = 1.2 * finance_datas['유동자산'] / finance_datas['자산총계'] \
                              + 1.4 * finance_datas['유동부채'] / finance_datas['자산총계'] \
                              + 3.3 * finance_datas['영업이익(손실)']  / finance_datas['자산총계'] \
                              + 0.6 * finance_datas['MARKETCAP'] / finance_datas['부채총계'] \
                              + 1.0 * finance_datas['수익(매출액)'] / finance_datas['자산총계']
finance_datas['1-부채비율'] = 1 - finance_datas["부채총계"] / finance_datas["자본총계"] # 역순을 위하여 1 에서 빼줌
    
finance_datas['PER'] = finance_datas['MARKETCAP'] / finance_datas['당기순이익(손실)']
finance_datas['PSR'] = finance_datas['MARKETCAP'] / finance_datas['수익(매출액)']
finance_datas['PBR'] = finance_datas['MARKETCAP'] / (finance_datas['자산총계'] - finance_datas['부채총계'])

finance_datas

,종목코드,년도,분기,배당금지급,유동자산,유동부채,영업이익(손실),수익(매출액),당기순이익(손실),자본총계,...,MARKETCAP,TSF_SLOPE,배당수익률,순이익률,자본회전율,AltmanZScore,1-부채비율,PER,PSR,PBR
0,000020,2020.0,1,3.312928e+09,2.379160e+11,5.601470e+10,2.706892e+09,6.700367e+10,2.259000e+09,2.994870e+11,...,1.829511e+11,-0.332,0.018108,0.033715,0.223728,2.600444,0.743900,80.987644,2.730463,0.610881
1,000950,2020.0,1,0.000000e+00,4.445098e+10,2.139210e+11,-3.642227e+09,2.840698e+10,-7.534318e+09,1.237910e+11,...,1.940399e+10,-0.329,0.000000,-0.265228,0.229475,0.998174,-1.126584,-2.575415,0.683071,0.156749
2,001540,2020.0,1,0.000000e+00,1.473480e+11,5.060352e+10,2.657248e+09,3.777548e+10,2.329921e+09,1.491430e+11,...,1.237726e+11,-0.310,0.000000,0.061678,0.253284,2.543271,0.571581,53.123087,3.276533,0.829896
3,001790,2020.0,1,6.212995e+09,5.346720e+11,4.020430e+11,7.324572e+09,2.836740e+11,1.456410e+08,4.575730e+11,...,1.412721e+11,-0.184,0.043979,0.000513,0.619954,1.433388,-0.520033,970.002003,0.498008,0.308741
4,002230,2020.0,1,0.000000e+00,9.711871e+10,2.172407e+10,-3.697193e+08,1.134262e+10,-2.315460e+09,1.336150e+11,...,5.249453e+10,-0.415,0.000000,-0.204138,0.084890,2.170490,0.801800,-22.671312,4.628076,0.392880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,383220,2023.0,3,6.102839e+10,7.170640e+11,4.755630e+11,1.485240e+11,4.928150e+11,1.190810e+11,1.221050e+12,...,4.328701e+12,0.004,0.014099,0.241634,0.403599,6.013897,0.538396,36.350893,8.783622,3.545067
3342,405100,2023.0,3,5.737761e+09,7.099587e+10,8.948139e+09,1.729899e+09,1.500569e+10,1.754362e+09,1.018220e+11,...,1.580033e+11,0.058,0.036314,0.116913,0.147372,4.224467,0.719246,90.063126,10.529557,1.551759
3343,425040,2023.0,3,4.249243e+09,1.721390e+11,6.219917e+10,3.685606e+09,2.418480e+10,4.575588e+09,1.681180e+11,...,4.170749e+11,-0.301,0.010188,0.189193,0.143856,4.889058,0.576001,91.152190,17.245329,2.480844
3344,900070,2023.0,3,4.103465e+06,5.672710e+08,1.290003e+08,9.139425e+06,1.489769e+08,4.868978e+06,6.494237e+08,...,3.342875e+10,-0.169,0.000123,0.032683,0.229399,108.326015,0.711581,6865.660103,224.388821,51.474478


In [22]:
# 데이터 중간 저장

finance_datas.to_csv('data/data_preprocessing_unified_including_others.csv', sep=',', encoding='utf-8', index=False)

finance_datas

,종목코드,년도,분기,배당금지급,유동자산,유동부채,영업이익(손실),수익(매출액),당기순이익(손실),자본총계,...,MARKETCAP,TSF_SLOPE,배당수익률,순이익률,자본회전율,AltmanZScore,1-부채비율,PER,PSR,PBR
0,000020,2020.0,1,3.312928e+09,2.379160e+11,5.601470e+10,2.706892e+09,6.700367e+10,2.259000e+09,2.994870e+11,...,1.829511e+11,-0.332,0.018108,0.033715,0.223728,2.600444,0.743900,80.987644,2.730463,0.610881
1,000950,2020.0,1,0.000000e+00,4.445098e+10,2.139210e+11,-3.642227e+09,2.840698e+10,-7.534318e+09,1.237910e+11,...,1.940399e+10,-0.329,0.000000,-0.265228,0.229475,0.998174,-1.126584,-2.575415,0.683071,0.156749
2,001540,2020.0,1,0.000000e+00,1.473480e+11,5.060352e+10,2.657248e+09,3.777548e+10,2.329921e+09,1.491430e+11,...,1.237726e+11,-0.310,0.000000,0.061678,0.253284,2.543271,0.571581,53.123087,3.276533,0.829896
3,001790,2020.0,1,6.212995e+09,5.346720e+11,4.020430e+11,7.324572e+09,2.836740e+11,1.456410e+08,4.575730e+11,...,1.412721e+11,-0.184,0.043979,0.000513,0.619954,1.433388,-0.520033,970.002003,0.498008,0.308741
4,002230,2020.0,1,0.000000e+00,9.711871e+10,2.172407e+10,-3.697193e+08,1.134262e+10,-2.315460e+09,1.336150e+11,...,5.249453e+10,-0.415,0.000000,-0.204138,0.084890,2.170490,0.801800,-22.671312,4.628076,0.392880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,383220,2023.0,3,6.102839e+10,7.170640e+11,4.755630e+11,1.485240e+11,4.928150e+11,1.190810e+11,1.221050e+12,...,4.328701e+12,0.004,0.014099,0.241634,0.403599,6.013897,0.538396,36.350893,8.783622,3.545067
3342,405100,2023.0,3,5.737761e+09,7.099587e+10,8.948139e+09,1.729899e+09,1.500569e+10,1.754362e+09,1.018220e+11,...,1.580033e+11,0.058,0.036314,0.116913,0.147372,4.224467,0.719246,90.063126,10.529557,1.551759
3343,425040,2023.0,3,4.249243e+09,1.721390e+11,6.219917e+10,3.685606e+09,2.418480e+10,4.575588e+09,1.681180e+11,...,4.170749e+11,-0.301,0.010188,0.189193,0.143856,4.889058,0.576001,91.152190,17.245329,2.480844
3344,900070,2023.0,3,4.103465e+06,5.672710e+08,1.290003e+08,9.139425e+06,1.489769e+08,4.868978e+06,6.494237e+08,...,3.342875e+10,-0.169,0.000123,0.032683,0.229399,108.326015,0.711581,6865.660103,224.388821,51.474478


In [23]:
# percentile (백분위) 정보 입력

selected_column_names = [ "ATR",
                          "배당수익률", "순이익률", "자본회전율",
                          "AltmanZScore", "1-부채비율",
                          "PER", "PSR", "PBR",
                          "TSF_SLOPE"]

for column_name in selected_column_names:
    percentile_rank = finance_datas[column_name].rank(pct=True)
    finance_datas[column_name + "_percentile"] = percentile_rank
    
print(finance_datas)

        종목코드      년도  분기         배당금지급          유동자산          유동부채  \
0     000020  2020.0   1  3.312928e+09  2.379160e+11  5.601470e+10   
1     000950  2020.0   1  0.000000e+00  4.445098e+10  2.139210e+11   
2     001540  2020.0   1  0.000000e+00  1.473480e+11  5.060352e+10   
3     001790  2020.0   1  6.212995e+09  5.346720e+11  4.020430e+11   
4     002230  2020.0   1  0.000000e+00  9.711871e+10  2.172407e+10   
...      ...     ...  ..           ...           ...           ...   
3341  383220  2023.0   3  6.102839e+10  7.170640e+11  4.755630e+11   
3342  405100  2023.0   3  5.737761e+09  7.099587e+10  8.948139e+09   
3343  425040  2023.0   3  4.249243e+09  1.721390e+11  6.219917e+10   
3344  900070  2023.0   3  4.103465e+06  5.672710e+08  1.290003e+08   
3345  950130  2023.0   3  2.275586e+07  3.739753e+08  3.345150e+07   

          영업이익(손실)       수익(매출액)     당기순이익(손실)          자본총계  ...  \
0     2.706892e+09  6.700367e+10  2.259000e+09  2.994870e+11  ...   
1    -3.642227e+09  2

In [24]:
# 백분위 정보에 대한 평균 값 구하기
# 추후에 세부 지표들에 대한 선택이 아닌 대분류에 대한 선택을 바탕으로 설명 모델의 성능 비교를 진행할 것이다.

finance_datas["avg_of_변동성지표_percentile"] = finance_datas["ATR_percentile"]

finance_datas["avg_of_수익성지표_percentile"] = (finance_datas["배당수익률_percentile"]
                                               + finance_datas["순이익률_percentile"]
                                               + finance_datas["자본회전율_percentile"]) / 3

finance_datas["avg_of_안정성지표_percentile"] = (finance_datas["AltmanZScore"]
                                               + finance_datas["1-부채비율"]) / 2

finance_datas["avg_of_가치성지표_percentile"] = (finance_datas["PER_percentile"]
                                               + finance_datas["PSR_percentile"]
                                               + finance_datas["PBR_percentile"]) / 3

finance_datas["avg_of_모멘텀지표_percentile"] = finance_datas["TSF_SLOPE_percentile"]


# 평균 값에 대한 새로운 백분위 값 구하기

selected_column_names = [ "avg_of_변동성지표_percentile", 
                          "avg_of_수익성지표_percentile", 
                          "avg_of_안정성지표_percentile",
                          "avg_of_가치성지표_percentile", 
                          "avg_of_모멘텀지표_percentile" ]

for column_name in selected_column_names:
    percentile_rank = finance_datas[column_name].rank(pct=True)
    finance_datas[column_name.replace("avg_of_", "")] = percentile_rank

In [25]:
finance_datas.to_csv('data/data_preprocessing_unified_including_others_and_percentile.csv', sep=',', encoding='utf-8', index=False)

finance_datas

,종목코드,년도,분기,배당금지급,유동자산,유동부채,영업이익(손실),수익(매출액),당기순이익(손실),자본총계,...,avg_of_변동성지표_percentile,avg_of_수익성지표_percentile,avg_of_안정성지표_percentile,avg_of_가치성지표_percentile,avg_of_모멘텀지표_percentile,변동성지표_percentile,수익성지표_percentile,안정성지표_percentile,가치성지표_percentile,모멘텀지표_percentile
0,000020,2020.0,1,3.312928e+09,2.379160e+11,5.601470e+10,2.706892e+09,6.700367e+10,2.259000e+09,2.994870e+11,...,0.533622,0.415920,1.672172,0.581092,0.039450,0.533622,0.285415,0.592947,0.636880,0.039450
1,000950,2020.0,1,0.000000e+00,4.445098e+10,2.139210e+11,-3.642227e+09,2.840698e+10,-7.534318e+09,1.237910e+11,...,0.617155,0.119396,-0.064205,0.122335,0.041094,0.617155,0.016139,0.037657,0.032277,0.041094
2,001540,2020.0,1,0.000000e+00,1.473480e+11,5.060352e+10,2.657248e+09,3.777548e+10,2.329921e+09,1.491430e+11,...,0.720114,0.279289,1.557426,0.631600,0.049163,0.720114,0.092947,0.545726,0.713389,0.049163
3,001790,2020.0,1,6.212995e+09,5.346720e+11,4.020430e+11,7.324572e+09,2.836740e+11,1.456410e+08,4.575730e+11,...,0.477286,0.591452,0.456677,0.399382,0.154961,0.477286,0.686940,0.090556,0.347878,0.154961
4,002230,2020.0,1,0.000000e+00,9.711871e+10,2.172407e+10,-3.697193e+08,1.134262e+10,-2.315460e+09,1.336150e+11,...,0.684250,0.045776,1.486145,0.328850,0.014644,0.684250,0.003288,0.516736,0.255828,0.014644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,383220,2023.0,3,6.102839e+10,7.170640e+11,4.755630e+11,1.485240e+11,4.928150e+11,1.190810e+11,1.221050e+12,...,0.505081,0.643853,3.276147,0.846683,0.498057,0.505081,0.811417,0.883144,0.921100,0.498057
3342,405100,2023.0,3,5.737761e+09,7.099587e+10,8.948139e+09,1.729899e+09,1.500569e+10,1.754362e+09,1.018220e+11,...,0.663628,0.545029,2.471857,0.843893,0.611327,0.663628,0.572475,0.795278,0.920203,0.611327
3343,425040,2023.0,3,4.249243e+09,1.721390e+11,6.219917e+10,3.685606e+09,2.418480e+10,4.575588e+09,1.681180e+11,...,0.786312,0.451783,2.732529,0.909942,0.052451,0.786312,0.351166,0.829946,0.962343,0.052451
3344,900070,2023.0,3,4.103465e+06,5.672710e+08,1.290003e+08,9.139425e+06,1.489769e+08,4.868978e+06,6.494237e+08,...,0.533622,0.247061,54.518798,0.996712,0.176330,0.533622,0.069038,0.996414,0.997609,0.176330
